In [ ]:
import json
import gzip
import pandas as pd
import re


In [31]:

# Importing metadata zip file and converting it to dataframe

def getDF(path):
  # Unzip the file, load in each line as an object
  g = gzip.open(path, 'rb')
  g = [json.loads(l) for l in g]

  # Map to a dictionary, then load in as a dataframe
  dict_df = {i: d for (i, d) in enumerate(g)}
  return pd.DataFrame.from_dict(dict_df, orient='index')

df = getDF('meta_ALL_Beauty.json.gz')

In [32]:
# Identify all columns, their sizes and datatypes in json file 

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32892 entries, 0 to 32891
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   category         32892 non-null  object
 1   tech1            32892 non-null  object
 2   description      32892 non-null  object
 3   fit              32892 non-null  object
 4   title            32892 non-null  object
 5   also_buy         32892 non-null  object
 6   tech2            32892 non-null  object
 7   brand            32892 non-null  object
 8   feature          32892 non-null  object
 9   rank             32892 non-null  object
 10  also_view        32892 non-null  object
 11  details          32892 non-null  object
 12  main_cat         32892 non-null  object
 13  similar_item     32892 non-null  object
 14  date             32892 non-null  object
 15  price            32892 non-null  object
 16  asin             32892 non-null  object
 17  imageURL         32892 non-null  obj

In [33]:

# Selecting only the colums that are required for analysis

colums_description_asin = ["description","asin"]
df = df[colums_description_asin]

df.describe()

,description,asin
count,32892,32892
unique,13751,32488
top,[],B00027CDOW
freq,17773,2


In [ ]:
# TODO: check fror NULL both description and asin

In [34]:

# Descriptions in the json file are a list of strings, converting to sting for easy cleaning
df["newdescription"] = df.description.map(lambda x: ".".join(x).replace("\n",""))


In [35]:
# Identify range of values in descriptions

# Excluding empty descriptions so can see the statistics about length of descriptions
df_descriptions_without_empty = df.drop(df[df["newdescription"] == ""].index)

# Identify range of values in descriptions
df_descriptions_without_empty = df_descriptions_without_empty["newdescription"].apply(lambda x: len(x))
df_descriptions_without_empty.describe()


count    15108.000000
mean       429.378740
std        886.166717
min          1.000000
25%         84.000000
50%        250.000000
75%        511.000000
max      34309.000000
Name: newdescription, dtype: float64

In [36]:
# Check for innapropriate characters in description column

# Check for non alphabetic or alphanumeric instances in description using Regex (in each description)
df["characters_in_description"] = df["newdescription"].apply(lambda x: set(re.findall(r'[^A-Za-z0-9\s]', x)))
df


,description,asin,newdescription,characters_in_description
0,[Loud 'N Clear Personal Sound Amplifier allows...,6546546450,Loud 'N Clear Personal Sound Amplifier allows ...,"{,, ., '}"
1,[No7 Lift & Luminate Triple Action Serum 50ml ...,7178680776,No7 Lift & Luminate Triple Action Serum 50ml b...,{&}
2,[No7 Stay Perfect Foundation now stays perfect...,7250468162,No7 Stay Perfect Foundation now stays perfect ...,"{!, ., -, ', ,, ;}"
3,[],7367905066,,{}
4,[Lacto Calamine Skin Balance Daily Nourishing ...,7414204790,Lacto Calamine Skin Balance Daily Nourishing L...,"{), -, (}"
...,...,...,...,...
32887,[],B01HIWLLUK,,{}
32888,[],B01HJ1K3YK,,{}
32889,[],B01HJ84SGM,,{}
32890,[Brand new and high quality<br> Enables fast v...,B01HJASD20,Brand new and high quality<br> Enables fast vo...,"{:, ), ., -, <, >, (}"


In [42]:
unnecessary_characters = set()
for x in df["characters_in_description"]:
    unnecessary_characters = unnecessary_characters.union(x)
print("All unecessy characters in description column:", unnecessary_characters)

All unecessy characters in description column: {'/', '|', '.', ']', '-', '<', '[', '%', '^', '=', '!', '#', '$', '&', '"', ',', '\\', ';', '~', ')', '+', "'", '>', '(', '?', '@', ':', '*', '_', '`', '{', '}'}


In [37]:
# Identify top words that come up in descriptions 

from collections import Counter
Counter(" ".join(df["newdescription"]).split()).most_common(100)

[('and', 33260),
 ('the', 26118),
 ('to', 19363),
 ('a', 16317),
 ('of', 15952),
 ('for', 12935),
 ('is', 11389),
 ('with', 10951),
 ('in', 9179),
 ('your', 9085),
 ('you', 6008),
 ('skin', 5348),
 ('or', 5324),
 ('that', 5148),
 ('on', 4552),
 ('The', 4460),
 ('are', 4154),
 ('it', 3666),
 ('<br>', 3513),
 ('hair', 3462),
 ('as', 3451),
 ('from', 3312),
 ('-', 3137),
 ('can', 2978),
 ('be', 2931),
 ('This', 2861),
 ('by', 2823),
 ('will', 2643),
 ('this', 2529),
 ('&', 2461),
 ('an', 2330),
 ('all', 2239),
 ('not', 2111),
 ('/>', 2075),
 ('natural', 1938),
 ('It', 1934),
 ('use', 1905),
 ('our', 1848),
 ('has', 1823),
 ('more', 1608),
 ('<br', 1598),
 ('have', 1510),
 ('any', 1467),
 ('A', 1442),
 ('<td', 1405),
 ('at', 1378),
 ('x', 1343),
 ('oil', 1323),
 ('up', 1304),
 ('skin.', 1262),
 ('For', 1261),
 ('1', 1234),
 ('product', 1232),
 ('also', 1223),
 ('body', 1216),
 ('used', 1190),
 ('which', 1183),
 ('color', 1178),
 ('one', 1166),
 ('help', 1127),
 ('while', 1112),
 ('its', 10

Large number of stopwords and characters identified. There are html tags used as well.

In [11]:
# ASIN analysis

asin = df["asin"].apply(lambda x: len(x))

# Check for lengths of asins
asin.describe()     

count    32892.0
mean        10.0
std          0.0
min         10.0
25%         10.0
50%         10.0
75%         10.0
max         10.0
Name: asin, dtype: float64

In [46]:
# Check for innapropriate characters in asin column

import re

# Check for non alphabetic(uppercase) or alphanumeric instances in the whole asin column using Regex
df["characters_in_asin"] = df["asin"].apply(lambda x: set(re.findall(r'[^A-Z0-9\s]', x)))

unnecessary_asin_characters = set()
for x in df["characters_in_asin"]:
    unnecessary_asin_characters = unnecessary_asin_characters.union(x)
print("There are no unecessy characters in asin column:", unnecessary_asin_characters)


There are no unecessy characters in asin column: set()
